In [10]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from tqdm.notebook import tqdm
import numpy as np
import json
import os
import os.path as osp

In [2]:
from dataset import MultiModalDataset
from model import FoodItemTagModel

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models, transforms

In [4]:
# base_path = "/Users/santhosh.mohan/Downloads/DSCVAssessment/assignments/food_item_tag"
base_path = "C:\\Users\\Mercedez\\Downloads\\santhosh\\food_item_tag"

In [5]:
image_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [6]:
def train_epoch(model, dataloader, loss_func, optim, device = 'cpu'):
    model.train()
    epoch_loss = 0
    total_size = 0
    for batch in tqdm(dataloader):
        image = batch['image'].to(device)
        price = batch['price'].to(device)
        label = batch['label'].to(device)
        text = {}
        text['input_ids'] = batch['input_ids'].to(device)
        text['attention_mask'] = batch['attention_mask'].to(device)
        text['token_type_ids'] = batch['token_type_ids'].to(device)
        optim.zero_grad()
        with torch.set_grad_enabled(True):
            output = model(image,text,price)
            loss = loss_func(output, label)
            loss.backward()
            optim.step()
        epoch_loss += loss.item() * label.size(0)
        total_size += label.size(0)
    epoch_loss = epoch_loss / total_size
    print(f"Training Loss - {epoch_loss}")

def validate_epoch(model, dataloader, loss_func, device = 'cpu'):
    model.eval()
    epoch_loss = 0
    total_size = 0
    for batch in tqdm(dataloader):
        image = batch['image'].to(device)
        price = batch['price'].to(device)
        label = batch['label'].to(device)
        text = {}
        text['input_ids'] = batch['input_ids'].to(device)
        text['attention_mask'] = batch['attention_mask'].to(device)
        text['token_type_ids'] = batch['token_type_ids'].to(device)
        with torch.set_grad_enabled(False):
            output = model(image,text,price)
            loss = loss_func(output, label)
        epoch_loss += loss.item() * label.size(0)
        total_size += label.size(0)
    epoch_loss = epoch_loss / total_size
    print(f"Validation Loss - {epoch_loss}")
    return epoch_loss

In [7]:
def train_image_model(device='cpu', epochs = 10):
    model = FoodItemTagModel(512,512,48)
    model = model.to(device)
    ### loading computed class weights
    with open(f"{base_path}\\data\\class_weights.json") as fh:
        line = fh.readline()
        weights = json.loads(line)
    weights = torch.tensor(weights)
    loss_func = nn.BCEWithLogitsLoss(pos_weight=weights)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    scheduler = lr_scheduler.MultiplicativeLR(optimizer,lr_lambda=lambda epoch: 0.95)
    training_dataset = MultiModalDataset(f"{base_path}\\data\\training_data.csv", f"{base_path}\\imgs", image_transforms["train"])
    validation_dataset = MultiModalDataset(f"{base_path}\\data\\validation_data.csv", f"{base_path}\\imgs", image_transforms["val"])


    training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=16,
                                                 shuffle=True, num_workers=0)
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=16,
                                                 shuffle=True, num_workers=0)#,collate_fn=lambda x: x)
    
    min_score = 1000.0
    for epoch in range(epochs):
        print(f"Epoch - {epoch+1}")
        train_epoch(model, training_dataloader, loss_func, optimizer, device)
        score = validate_epoch(model, validation_dataloader, loss_func, device)
        scheduler.step()
        if score < min_score:
            torch.save(model.state_dict(), f"model_{epoch}.pth")
            if osp.exists("best_model.pth"):
                os.remove("best_model.pth")
            os.rename( f"model_{epoch}.pth","best_model.pth")
        
    return model

In [8]:
model = train_image_model(epochs = 20,device='cuda:0')
torch.save(model.state_dict(), "model.pth")

Some weights of the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_le

Epoch - 1


  0%|          | 0/496 [00:00<?, ?it/s]

Training Loss - 0.6805232460808723


  0%|          | 0/124 [00:00<?, ?it/s]

...

Training Loss - 0.2113949682366501
Epoch - 20


  0%|          | 0/496 [00:00<?, ?it/s]

Training Loss - 0.21447927217451387


  0%|          | 0/124 [00:00<?, ?it/s]

Training Loss - 0.21143645663458907
